In [1]:
import os
import numpy as np
import math
import csv
import pandas as pd
import matplotlib.pyplot as plt
%config IPCompleter.greedy=True

In [2]:
#FIXME: перенести значения в файл настроек
Re = 250000
M = 0.0
ncr = 12
it = 1000
alpha_min = -3
alpha_max = 13
alpha_step = 0.2

In [3]:
#NEW!
#my lib
def isNaN(num):
    return num != num

In [4]:
#FIXME: что
def Re(V, l, nu = 1.5 * 10 ** (-5)):
    return V * l / nu

def CL_required (V, S, m, rho = 1.22):
    return (2 * m * 9.81 / (rho * V ** 2 * S))

def CDi (CL, AR, e = 0.9):
    return (CL ** 2 / (math.pi * e * AR))

In [5]:
#NEW!
#для создания своих исключений нужно написать класс и наследовать его от встроенного класса  Exception
#подробнее об исключениях можно прочитать тут
#https://pythonchik.ru/osnovy/python-try-except
class InsufficientInputData(Exception):
    def __init__(self, text):
        self.txt = text

In [6]:
class Foil():
    def __init__(self, name):
        pass
    def get_geometry(self):
        pass
    def get_filename(self):
        pass
    

In [13]:
#UPDATED!
class Solver():
    def __init__(self, performance_file_name, params_file_name, settings_file_name, foils_file_names, m):
        #UPDATED! self.performance - словарь
        df = pd.read_csv(performance_file_name, header=None)
        self.performance = dict(zip(df[0], df[1]))
        self.preprocess_performance()

        #UPDATED! self.settings - словарь
        df = pd.read_csv(settings_file_name, header=None)
        self.settings = dict(zip(df[0], df[1]))

        self.params=pd.read_csv(params_file_name, index_col=False).T
        
        self.foils = []
        for name in foils_file_names:
            self.foils(Foil(name))

        self.geometry = {}
        self.aero = {"CL_cr" : 0, "K_cr" : 0}
        self.m = m

        #NEW! макрос для создания переменных типа 'cy' со значением self.performance['cy']
        var_list=[]
        self.init_var_list_macros = f"for var in {var_list}:\n\texec('%s = %d' % (i,self.performance[var]))"

    #UPDATED!
    def wing_area(self):
        var_list = ["TOM", "take_off_speed", "CL_take_off", "g", "Density"]
        exec(self.init_var_list_macros)
        #FIXME дописать self
        return 2 * self.performance["TOM"] * self.settings["g"] / (Density * take_off_speed ** 2 * CL_take_off)
        
    #UDRATED!
    def CL_cruise(self):
        var_list = ["TOM", "cruise_speed", "wing_area", "g", "Density"]
        exec(self.init_var_list_macros)
        
        return 2 * TOM * g / (Density * cruise_speed ** 2 * wing_area)

    #UDRATED!
    def wingspan (self):
        var_list = ["AR", "wing_area"]
        exec(self.init_var_list_macros)
        
        return (AR * wing_area) ** 2

    #UDRATED!
    def ba(self):
        var_list = ["wing_area", "AR"]
        exec(self.init_var_list_macros)
        
        return (wing_area / AR) ** 0.5

    #UDRATED!
    def aft_area(self):
        var_list = ["A_aft", "wing_area", "ba", "l_stab"]
        exec(self.init_var_list_macros)
        
        return A_aft * wing_area * ba / l_stab

    #UDRATED!
    def keel_area(self):
        var_list = ["B_keel", "wing_area", "wingspan", "l_stab"]
        exec(self.init_var_list_macros)
        
        return B_keel * wing_area * wing_span / l_stab

    #UDRATED!
    def gamma(self):
        var_list = ["keel_area", "aft_area"]
        exec(self.init_var_list_macros)
        
        return math.arctan((keel_area / aft_area) ** 0.5)

    #UDRATED!
    def stab_area(self):
        var_list = ["aft_area", "gamma"]
        exec(self.init_var_list_macros)
        
        return aft_area / (math.cos(gamma)) ** 2

    #UDRATED!
    def P_cruise(self):
        var_list = ["TOM", "K_cruise", "eta_prop", "g"]
        exec(self.init_var_list_macros)
        
        return TOM * g / (K_cruise * eta_prop)

    #UDRATED!
    def wire_length(self):
        var_list = ["wingspan", "l_stab", "scale_coef"]
        exec(self.init_var_list_macros)
        
        return wingspan / 2 + 2 * scale_coef * l_stab


    def XFoil_command_CL(self): #(V, AR, Sw, m, it = 1000, M = 0.0, work_path = path, foil_name = foil):
        command_file=open(work_path + 'commands.in','w')
        command_file.write('load ' + work_path + foil_name + '.txt'+'\n'\
        + foil_name + '\n\
        panel\n\
        oper\n\
        visc ' + str(Re(V, ba(Sw, AR))) + '\n\
        M ' + str(M) + '\n\
        type 1\n\
        vpar\n\
        n ' + str(ncr) + '\n\
        \n\
        iter\n' + str(it) + '\n\
        pacc\n'\
        + work_path + 'polar.dat\n\
        \n\
        cl ' + str(CL_required(V, Sw, m)) +'\n\
        \n\
        \n\
        quit\n')
        command_file.close()
        return 0

    def XFoil_run(file_path = path, xfoil_path = "D:\Desktop\XFoil\\"):
        sep=os.path.sep
        run_xfoil_command = xfoil_path + 'xfoil < ' + file_path + \
        'commands.in'
        os.system(run_xfoil_command)
        return 0
    
    def XFoil_read(file_path = path, data_path = path):
        aero_data_file = open(file_path + 'polar.dat', 'r')
        lines = aero_data_file.readlines()
        aero_data_file.close()
        #delete Xfoil output file ready for next Xfoil run
        os.system('del ' + file_path + 'polar.dat')
        #Linux/OSX: os.system('rm -f ' + file_path + 'polar.dat')
        alpha = float(lines[-1][2: 8])
        cl = float(lines[-1][11: 17])
        cd = float(lines[-1][20: 27])
        return [alpha, cl, cd]
        
    def K_V_solver(V, AR, Sw, m):
        polar = np.array([])
        for v in V:
            XFoil_command_CL(v, AR, Sw, m)
            XFoil_run()
            polar = np.append(polar, [[XFoil_read()]])
            print(v)
        polar = polar.reshape(polar.size // 3, 3)
        return polar

    
    def AR_selection(self):
        pass

    #NEW!
    def preprocess_settings(self):
        #FIXME константы надо брать из settings
        if not self.settings['g']:
            self.settings['g'] = 9.81
            
        if not self.performance['scale_coef']:
            self.performance['scale_coef'] = 1.3
            
        if not self.settings['Density']:
            self.settings['Density'] = 1.22
        
        critical_params = ['TOM', 'take_off_speed', 'CL_take_off', 
                           'K_cruise', 'eta_prop', 'keel_area', 'aft_area', 
                           'cruise_speed', 'A_aft', 'B_keel', 'wing_area', 
                           'wingspan', 'ba', 'l_stab', 'AR', 'scale_coef']
        for param in critical_params:
            if isNaN(self.performance.get(param, np.nan)):
                raise InsufficientInputData(f'obligatory parametr {name} not found')

    #FIXME
    def calculate(self):        
        self.geometry["wing_area"] = self.wing_area()
        self.CL_cr = self.CL_cruise()
        self.AR, self.K_cr = self.AR_selection()
        self.geometry["wingspan"] = self.wingspan()
        self.geometry["tail_horizontal"] = self.aft_area()
        self.geometry["tail_vertical"] = self.keel_area()
        self.geometry["Vdihedral"] = self.gamma()
        self.geometry["Vtail_area"] = self.stab_area()
        self.geometry["Pcruise"] = self.P_cruise()
        self.geometry["wirelength"] = self.wire_length()
        
    def update_m(self, new_m):
        self.m = new_m
    
    def write_geometry(self):
        with open("geometry.csv", 'w') as f:
            for key in self.geomatry.keys():
                f.write("%s, %s\n" % (key, self.geomatry[key]))

    def get_data_to_weigh():
        return [self.params, self.geometry]

In [14]:
#NEW!
#m (масса ЛА) -- есть сумма
#слагаемые бывают трёх типов:
#1) произведение двух величин из params
#2) произведение двух величин -- одна из params, одна из geometry
#3) величина из params
def weigh(params_file_name, geometry_file_name):
    df = pd.read_csv('data.csv', header=None)
    geom = dict(zip(df[0], df[1]))
    params=pd.read_csv(params_file_name, index_col=False).T
    
    multiply_dict = dict()
    m = 0
    for var in params:
        if isNaN(params[var]["value"]):
            raise InsufficientInputData(f"no value for variable '{params[var]['name']}'")

        if not isNaN(params[var]["index"]):
            multiply_dict[params[var]["index"]] = multiply_dict.get(params[var]["index"], [])
            multiply_dict[params[var]["index"]].append(params[var]["value"])
        elif not isNaN(params[var]["conjugate name"]):
            if not isNaN(geom.get(params[var]["conjugate name"], np.nan)):
                m += geom[params[var]["conjugate name"]] * params[var]["value"]
            else:
                raise InsufficientInputData(f"for variable '{params[var]['name']}', '{params[var]['conjugate name']}' "+\
                                            f"is specified as conjugate, but it is no '{params[var]['conjugate name']}'"+\
                                            f" in the performance-file")
        elif not isNaN(params[var]["value"]):
            m += params[var]["value"]
    
    for vals in multiply_dict.values():
        m += math.prod(vals)
    return m
    
#weigh("params.csv", "data.csv")

In [9]:
#принимает необходимые даные и делает вывод, успешен ли расчёт 
#опирается на разницу m_target - m_получившаяся
def is_residual(...): 
    pass

SyntaxError: invalid syntax (3605395231.py, line 3)

In [10]:
# Модуль расчёта геометрических характеристик потребляет на вход: 
# dataframe PERFORMANCE с потребными эксплуатационными характеристиками
# dataframe PARAMS с параметрами электронных компонент, конструкционных материалов, аккумуляторов и т.д.
# пути до файлов селиговского формата с профилем крыла wing_foil и с профилем оперения aft_foil.
# TOM - значение взлётной массы в начальном приближении.

# PERFORMANCE включает параметры take_off_speed, cruise_speed, flight_time, payload

# PARAMS включает параметры m_FPV, m_powerplant, m_flight_control, m_fus, m_servo1, m_servo2,
# line_dens_wire, line_dens_tube1, line_dens_tube2, line_dens_tube3,
# area_dens_LWPLA
# energy_dens_bat
# number_servo1, number_servo2
# l_stab

# Все единицы в СИ
def geom_solver(PERFORMANCE, PARAMS, wing_foil, aft_foil, TOM):
    performance_file_name = input("имя файла performances: ")
    params_file_name = input("имя файла params: ")
    settings_file_name = input("имя файла settings: ") 
    foils_file_names = input("имена файлов с профилями (через запятую): ")
    m = input("масса перевозимого груза: ")
    max_iter = input("максимальное число итераций: ")
    freq_geom_saveing = input("частота сохранения промежуточных геометрии: ")
    
    solver = Solver(performance_file_name, params_file_name, settings_file_name, foils_file_names, m)
    i = 0
    while True:
        solver.calculate()
        weigh(solver.get_data_to_weigh())
        if not is_residual(): #FIXME
            solver.update_m()
        else:
            print(f"сошлось на итерации: {i}")
            solver.write_geometry()
            print("информация сохранена в файл с геометрией")
            break
        
        if i == max_iter:
            print(f"прошло {i} итераций, но расчёт всё ещё не завершён.")
            flag = 2
            while flag not in [0, 1]:
                flag = input(f"введите 1, чтобы произвести ещё {max_iter} операций, иначе 0: ")
            if flag:
                i = 0
            else:
                solver.write_geometry()
                print("последняя итерация геометрии сохранена")
                break
                
        if i%freq_geom_saveing == 0:
            solver.write_geometry()
        i++
    
    
    

SyntaxError: invalid syntax (1362929658.py, line 53)

In [11]:
path = "D:\Desktop\XFoil\SG_6041\\"
foil = "SG6041"

def XFoil_command_CL(V, AR, Sw, m, it = 1000, M = 0.0, work_path = path, foil_name = foil):
    command_file=open(work_path + 'commands.in','w')
    command_file.write('load ' + work_path + foil_name + '.txt'+'\n'\
    + foil_name + '\n\
    panel\n\
    oper\n\
    visc ' + str(Re(V, ba(Sw, AR))) + '\n\
    M ' + str(M) + '\n\
    type 1\n\
    vpar\n\
    n ' + str(ncr) + '\n\
    \n\
    iter\n' + str(it) + '\n\
    pacc\n'\
    + work_path + 'polar.dat\n\
    \n\
    cl ' + str(CL_required(V, Sw, m)) +'\n\
    \n\
    \n\
    quit\n')
    command_file.close()
    return 0

def XFoil_run(file_path = path, xfoil_path = "D:\Desktop\XFoil\\"):
    sep=os.path.sep
    run_xfoil_command = xfoil_path + 'xfoil < ' + file_path + \
    'commands.in'
    os.system(run_xfoil_command)
    return 0
    
def XFoil_read(file_path = path, data_path = path):
    aero_data_file = open(file_path + 'polar.dat', 'r')
    lines = aero_data_file.readlines()
    aero_data_file.close()
    #delete Xfoil output file ready for next Xfoil run
    os.system('del ' + file_path + 'polar.dat')
    #Linux/OSX: os.system('rm -f ' + file_path + 'polar.dat')
    alpha = float(lines[-1][2: 8])
    cl = float(lines[-1][11: 17])
    cd = float(lines[-1][20: 27])
    return [alpha, cl, cd]
    
def K_V_solver(V, AR, Sw, m):
    polar = np.array([])
    for v in V:
        XFoil_command_CL(v, AR, Sw, m)
        XFoil_run()
        polar = np.append(polar, [[XFoil_read()]])
        print(v)
    polar = polar.reshape(polar.size // 3, 3)
    return polar